# heatmap radial average vs pp_delay

In [12]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [13]:
import numpy as np  
import matplotlib.pyplot as plt
import h5py
import os
import src.pyabel_polar
import scipy

* create radial average form one .hdf5 file
* for different fragments -> ToF ranges
* connect with pump_probe delay -> file name
* save results as npy

In [14]:
def number_of_trains_from_hdf(hdf_file_complete_path):
    '''Retrun number of recorded FEL trains in HDF file'''
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        trains = len(h_file['tpx3Times/triggerNr'][:])
    print('Number of trains in HDF file: {:.2e}'.format(trains))
    return trains

def data_from_hdf(hdf_file_complete_path, event_type = 'raw'):
    '''Read data from TimePix HDF files
    Choose raw or centroided data, default is centroided data'''
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        tof = h_file[str(event_type)+'/tof'][:]
        x_pos = h_file[str(event_type)+'/x'][:]
        y_pos = h_file[str(event_type)+'/y'][:]
    print('Reading from HDF5 - number of events: {:.2e}'.format(len(tof)))
    return tof, x_pos, y_pos

def data_sliced_by_tof(hdf_file_complete_path, tof_start = 0 , tof_end = 0.1, event_type = 'raw'):
    '''Slice data with respect to time-of-flight dimension'''
    tof, x_pos, y_pos = data_from_hdf(hdf_file_complete_path, event_type)
    sliced_x_pos = x_pos[np.logical_and(tof > tof_start, tof < tof_end)]
    sliced_y_pos = y_pos[np.logical_and(tof > tof_start, tof < tof_end)]
    sliced_tof = tof[np.logical_and(tof > tof_start, tof < tof_end)]
    print('Slicing - number of events: {:.2e} | {:.2%}'.format(len(sliced_tof), len(sliced_tof)/len(tof)))
    return sliced_tof, sliced_x_pos, sliced_y_pos

def reduce_raw_data(tof, x_pos, y_pos, number_of_events):
    '''Reduce data for visualization'''
    return tof[:number_of_events], x_pos[:number_of_events],y_pos[:number_of_events]

def tof_conversion(tof, time_unit):
    '''Convert time axis'''
    if time_unit == None:
        return tof, 's'
    if time_unit == 'milli':
        return tof*10**3, 'ms'
    if time_unit == 'micro':
        return tof*10**6, 'us'

def plot_tof(tof, hist_bins=100, time_unit = None):     
    '''Plot time-of-flight spectrum via histogram'''
    fig = plt.subplots(num = 1)
    plt.clf()
    tof, time_tof_unit = tof_conversion(tof, time_unit)
    plt.hist(tof, bins = hist_bins)
    plt.title('histogram: time-of-flight')
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('number of events')
    plt.show()
        
def plot_1d_histograms(tof, x_pos, y_pos, time_unit = None):      
    '''1D plots - detector position vs time-of-flight for each dimension'''
    tof, time_tof_unit = tof_conversion(tof, time_unit)
    fig = plt.figure(num = 2)
    plt.clf()
    plt.plot(tof, x_pos,'.')
    plt.title("ToF vs x_pos")
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('x_pos [px]')
    
    fig = plt.figure(num=3)
    plt.clf()
    plt.plot(tof, y_pos,'.')
    plt.title("ToF vs y_pos")
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('y_pos [px]')
    plt.show()

def plot_2d_histograms(tof, x_pos, y_pos ,bin_tof = 2000, bin_space = 200, time_unit = None):
    '''2D plot (heatmap) - detector position vs time-of-flight for each dimension'''
    tof, time_tof_unit = tof_conversion(tof, time_unit)
    fig = plt.figure(num = 4)
    plt.clf()
    plt.hist2d(tof, x_pos, bins= (bin_tof,bin_space))
    plt.title('2d histogram: time-of-flight / x_pos')
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('x_pos')
    plt.colorbar()

    fig = plt.figure(num = 5)
    plt.clf()
    plt.hist2d(tof, y_pos,bins= (bin_tof,bin_space))
    plt.title('2d histogram: time-of-flight / y_pos')
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('y_pos')
    plt.show()
    plt.colorbar()
    
def vmi_image(x_pos, y_pos, show_image = True):
    '''Display VMI image - cmax empirically found to surpress hot pixel '''
    fig = plt.figure(num = 6)
    plt.clf()
    counts, xbins, ybins, image = plt.hist2d(x_pos, y_pos,bins= (255, 255), cmax= 1000)
    plt.colorbar()
    plt.xlabel('x_pos [px]')
    plt.ylabel('y_pos [px]')
    if show_image == False:
        plt.close()
    return counts
    
def display_tof_and_vmi_of_tof_interval(hdf_file_complete_path, tof_start = 0 , tof_end = 0.1, hist_bins = 100, time_unit = None, event_type = 'raw'):
    '''Display VMI Image and time-of-flight spetrum'''
    tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start, tof_end, event_type)
    plot_tof(tof, hist_bins, time_unit)
    vmi_image(x_pos,y_pos)

    
def transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius ):
    
    counts = vmi_image(x_pos, y_pos, show_image = False)
    image_cart = np.flipud(counts.transpose())
    
    image_polar, r_grid, theta_grid = src.pyabel_polar.reproject_image_into_polar(image_cart, origin=(x_center,y_center))
    radial_ave = np.sum(image_polar, axis=1)
    
    fig = plt.figure(figsize = (10,10),num = 6)
    plt.clf()
    plt.imshow(image_cart)
    plt.scatter(x_center, y_center, color='r')
    plt.gcf().gca().add_artist(plt.Circle((x_center, y_center), radius, color='r', fill=False))
    plt.xlabel('x_posr [px]')
    plt.ylabel('y_posr [px]')
    plt.title('VMI image')

#     fig = plt.figure(num = 7)
#     plt.clf()
#     plt.imshow(image_polar)
#     plt.title('Image - polar coordinates')

    fig = plt.figure(num = 8)
    plt.clf()
    plt.plot(radial_ave,'r-')
    plt.title('Radial average')
    plt.xlabel('r [px]')
    plt.ylabel('counts')
    plt.show()
    
    return radial_ave

### Define source file & describe measurment

In [36]:
file_prefix = 'delay_test'
fragment = 'I2+'
delay_stage = 64

In [37]:
hdf_file_path = '../data/'
run_number = 52

file_start = "run_"+str(run_number).zfill(4)
file_end = '.hdf5'
hdf_file = [i for i in os.listdir(hdf_file_path) if os.path.isfile(os.path.join(hdf_file_path,i)) and i.startswith(file_start) and i.endswith(file_end)][0]
hdf_file_complete_path = hdf_file_path+hdf_file
assert os.path.isfile(hdf_file_complete_path), 'File does not exist!'

### Complete time-of-flight

In [6]:
tof_start = 0E-6
tof_end = 12E-6
tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start , tof_end)

plot_tof(tof, hist_bins = 500, time_unit = 'micro')

Reading from HDF5 - number of events: 1.50e+07
Slicing - number of events: 1.41e+07 | 94.58%


### Slice time-of-flight dimension and create VMI image

In [7]:
tof_start = 7.6E-6
tof_end = 8.04E-6

display_tof_and_vmi_of_tof_interval(hdf_file_complete_path, tof_start, tof_end, time_unit = 'micro')

Reading from HDF5 - number of events: 1.50e+07
Slicing - number of events: 2.09e+05 | 1.40%


### Chose center and calc radial average

In [38]:
# x_center, y_center = 130, 120        #I2+
# radius = 18

x_center, y_center = 130, 120
radius = 18

tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start , tof_end)
radial_average = transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius)

Reading from HDF5 - number of events: 2.43e+07
Slicing - number of events: 3.18e+05 | 1.31%


### Save to intermediate result

In [39]:
save_to_path = 'intermediate_data/'
number_of_trains = number_of_trains_from_hdf(hdf_file_complete_path)

save_file_name = str(file_prefix)+"_"+str(fragment)+"_"+str(delay_stage)
np.savez(save_to_path+save_file_name, radial_average=radial_average, fragment= fragment, delay_stage = delay_stage, number_of_trains = number_of_trains)

print(save_file_name)

Number of trains in HDF file: 6.56e+03
delay_test_I2+_64


___
# Create heatmap

* sort by pp-delay
* create heatmap

In [40]:
load_from_path = 'intermediate_data/'
file_prefix = 'delay_test'
fragment = 'I2+'

files = [i for i in os.listdir(load_from_path) if os.path.isfile(os.path.join(load_from_path,i)) and str(file_prefix)+"_"+str(fragment) in i]

print(files)

['delay_test_I2+_60.npz', 'delay_test_I2+_62.npz', 'delay_test_I2+_61.npz', 'delay_test_I2+_63.npz', 'delay_test_I2+_64.npz']


In [61]:
radial_averages =[]
delay_stage_pos = []
norm_counts = []

for i in files:
    npzfile = np.load(load_from_path+i)
    radial_averages.append(npzfile['radial_average'])
    delay_stage_pos.append(float(npzfile['delay_stage']))
    fragment = (npzfile['fragment'])
    norm_counts.append(float(npzfile['number_of_trains']))
    
radial_averages = np.array(radial_averages)
delay_stage_pos = np.array(delay_stage_pos)
norm_counts = np.array(norm_counts)

sort_ind = np.argsort(delay_stage_pos)
radial_averages = radial_averages[sort_ind]
norm_counts = norm_counts[sort_ind]
delay_stage_pos = delay_stage_pos[sort_ind]

print('fragment:',fragment) 
print('number of delay_stage_pos:',len(delay_stage_pos))
print('delay_stage_steps:',delay_stage_pos)


fragment: I2+
number of delay_stage_pos: 5
delay_stage_steps: [60. 61. 62. 63. 64.]


In [62]:
fig = plt.figure(figsize = (10,10),num = 10)


for i in range(len(radial_averages)):
    plt.plot(radial_averages[i],label=delay_stage_pos[i])
    print(len(radial_averages[i]))
plt.xlabel('r')
plt.ylabel('counts')
plt.legend()
plt.title(str(fragment));

187
187
187
187
187


5

In [113]:
tt = np.column_stack(radial_averages)

In [120]:
plt.imshow(tt, aspect="auto", origin="lower")
plt.xticks(np.arange(5), delay_stage_pos)

([<matplotlib.axis.XTick at 0x7f5c829a1e80>,
 <a list of 5 Text xticklabel objects>)